# Scraping Maryland Business Licenses with Selenium

Maryland has a [great portal](https://jportal.mdcourts.gov/license/pbPublicSearch.jsp) for searching business licenses, but the only problem is you have to check a box in order to get in.

1. Try to visit [the public search page](https://jportal.mdcourts.gov/license/pbPublicSearch.jsp)
2. Get redirected to a "I agree to this" page. Click that you've read the disclaimer, click Enter the Site.
3. Click "Search License Records" down at the bottom of the page
4. You're now on the search page! From the "Jurisdiction" dropdown, select "Statewide"
5. In the "Trade Name" field, type "Vap%" to try to find vape shops
6. Click "Next" in the bottom right-hand corner to go to the next page
7. Click "Click for detail" to see the details for a specific business license.

That's a lot of stuff! **Let's get to work.**

## Import what you need

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait

import time

## Preparation

### When you search for a business license, what URL should Selenium try to visit first?

In [2]:
driver = webdriver.Chrome()

In [3]:
driver.get("https://jportal.mdcourts.gov/license/pbPublicSearch.jsp")

**It isn't going to work, though! It's going to redirect to that intro page.** You can use *Incognito mode* to go back through the "Check the box, etc" series of pages, or you can close and re-open Chrome.

- Check the checkbox, then submit the form to accept their terms of service

Selenium can submit forms by either

- Selecting the form and using `.submit()`, or
- Selecting the button and using `.click()`

You only need to be able to get **one, not both.**

- *TIP: if something doesn't have anything special about it, xpath might be your best bet*

In [4]:
checkbox = driver.find_element_by_xpath('//*[@id="checkbox"]')
checkbox.click()
enter = driver.find_element_by_xpath('/html/body/table/tbody/tr[7]/td/form/div/input[2]')
enter.click()

Now click the **Search License Records** link up top in the navigation to get to the search page.

- *TIP: Honestly you could also just visit the URL directly now since you filled out that terms of service thing*

In [5]:
driver.get("https://jportal.mdcourts.gov/license/pbPublicSearch.jsp")

### Perform your search

Pick "Statewide" for the jurisdiction dropdown, and `VAP%` into the Trade Name field. The `%` is a wildcard.

In [6]:
from selenium.webdriver.support.ui import Select

juristiction = Select(driver.find_element_by_name("slcJurisdiction"))
juristiction.select_by_visible_text("Statewide")

trade_name = driver.find_element_by_name("txtTradeName")
trade_name.send_keys("VAP%")

submit_button = driver.find_element_by_xpath("/html/body/table[2]/tbody/tr[4]/td[2]/form/table/tbody/tr[14]/td/input[1]")
submit_button.submit()

And now, of course, **submit the form**.

- *TIP: Since scrolling to buttons can be a pain, sometimes it's easier to select the form and use `.submit()` instead of `.click()`ing the button*

## (Try to) scrape the results

Let's start by just **printing this stuff**. We'll save it as a dataframe later on.

For now, just scrape **each store's name**, then cry a little. Fact: this is an impossible and miserable page. 

In [8]:
results = driver.find_elements_by_class_name("searchfieldtitle")
for result in results:
    name = result.find_elements_by_tag_name("td")[1].text

    print(name)

VAPE IT STORE II
VAPE IT STORE I
VAPEPAD THE
VAPE FROG
VAPE FROG


To avoid struggling with the search results page, we're going to use the **detail page** instead. Try to figure out how to select it and click it inside of your `for` loop.

- *TIP: Instead of just looking for an `a` or an `img`, you might want to look for one of its parents first, then click. This might affect the way you print out the shop's name, too*
- *TIP: Not all of them have links! You can wrap in try/except to skip it, or you can check to see if the shop's status is Pending.*

In [40]:
results = driver.find_elements_by_class_name("searchfieldtitle")
count = 0

while count < 5:
    name = results[count].find_elements_by_tag_name("td")[1].text
    if results[count].find_elements_by_tag_name("td")[2].text != "Pending *":
        detail_button = results[count].find_elements_by_tag_name("td")[2].find_element_by_tag_name("a")
        detail_button.click()
        print("this works for", name)
        driver.execute_script("window.history.go(-1)")
        time.sleep(2)
    else:
        print("link not available for", name)
    count = count + 1
    results = driver.find_elements_by_class_name("searchfieldtitle")

this works for VAPE IT STORE II
this works for VAPE IT STORE I
this works for VAPEPAD THE
link not available for VAPE FROG
link not available for VAPE FROG


Okay, now let's get to action. For each result, **click the link to the detail page** and print out the following information:

- Mailing address
- Location address
- License information (you can keep it as one field)
- Total amount paid
- Issued by
- If you're feeling crazy, get the licenses, too.

If it doesn't have a detail page, just print out the name and that's all we need.

- *TIP: When you're done getting the information, you probably want to click back to the search results*
- *TIP: You might enjoy `find_element_by_partial_link_text` to do that*
- *TIP: Licenses can be acquired by doing some really odd list slicing - think about where it starts and where it ends, relative to the beginning and end of everything.*

> **IMPORTANT NOTE:** This is doomed. It's useful to do, but your current process is doomed. Once you get a `stale element reference` error move on to the next cell.

In [123]:

# Mailing address
mailing = driver.find_elements_by_class_name("tablecelltext")[0].find_elements_by_tag_name("td")[0].text
print("mailing:", mailing)
print("----------")

# Location address
location = driver.find_elements_by_class_name("tablecelltext")[1].find_elements_by_tag_name("td")[0].text
print("location:", location)
print("----------")

# License information (you can keep it as one field)
license_info = driver.find_elements_by_class_name("tablecelltext")[0].find_elements_by_tag_name("td")[1].text
print("license_info:", license_info)
print("----------")

# Total amount paid
a_paid = driver.find_elements_by_class_name("tablecelltext")[-3].find_elements_by_tag_name("td")[1].text
print("total amount paid:", a_paid)
print("----------")

# Issued by
issued_by = driver.find_elements_by_class_name("tablecelltext")[-2].text
print("issued by:", issued_by)
print("----------")

# If you're feeling crazy, get the licenses, too.
license_detail = []
licenses = driver.find_elements_by_tag_name("table")[-4].find_elements_by_class_name("tablecelltext")
for license in licenses[:-3]:
    license_d = license.text.split(" ")[2:-2]
    license_c = " ".join(license_d)
    license_detail.append(license_c)
print(license_detail)

mailing: AMIN NARGIS
1104 PLANTERS PLACE
SALISBURY, MD 21804
----------
location: VAPE IT STORE I
1724 N SALISBURY BLVD UNIT 2
SALISBURY, MD 21801
----------
license_info: License Status: Issued
License No.: 22375605
Control No.: 22591855
Date of Issue: 4/27/2018
Months Paid: 12
Exp. Date: 4/30/2019
Subdivision: 22 Salisbury
----------
total amount paid: $ 24.00
----------
issued by: WICOMICO COUNTY, CLERK OF CIRCUIT COURT
P.O. BOX 198
SALISBURY, MARYLAND 21803-0198
----------
["TRADER'S LICENSE", 'CHAIN STORE']


### Stale message reference

Once you navigate away from a page, and you go back to it, you can't use the variables from the first time you were on the page. So, we got a list of results when we first visited, clicked to the details page, clicked back, and now our original list is "stale."

This is sad.

Let's try this again: loop through the results and create a dataframe with `name` and `url` columns. And yes, some of them won't have URLs.

In [94]:
#I got around this with while loop, sort of
#still
import pandas as pd

info_list = []
nolink_list = []
results = driver.find_elements_by_class_name("searchfieldtitle")
for result in results:
    info_dic = {}
    name = result.find_elements_by_tag_name("td")[1].text
    try:
        url = result.find_elements_by_tag_name("td")[2].find_element_by_tag_name("a").get_attribute("href")
        info_dic["name"] = name
        info_dic["url"] = url
        info_list.append(info_dic)
    except:
        nolink_list.append(name)
        
df = pd.DataFrame(info_list)
print(df)
print("-------------")
print(nolink_list)

df.url

                name                                                url
0  VAPORFI SOUTHSIDE  https://jportal.mdcourts.gov/license/pbLicense...
1         VAPE DO JO  https://jportal.mdcourts.gov/license/pbLicense...
2         VAPERTEKMD  https://jportal.mdcourts.gov/license/pbLicense...
-------------
['VAPERTETMD', 'VAPORIOT']


0    https://jportal.mdcourts.gov/license/pbLicense...
1    https://jportal.mdcourts.gov/license/pbLicense...
2    https://jportal.mdcourts.gov/license/pbLicense...
Name: url, dtype: object

### Getting all of the results

After you've looped through the results on one page, we're going to want to go to the next page! Add a line to make it click the 'Next' button down at the bottom

- *Tip: `find_element_by_partial_link_text` will be your friend*
- *Tip: You might need to do the scrolling thing to get it onto the screen (and by that I mean, you WILL need to, so you should)*

Confirm that it moves to the next page (it doesn't need to scrape anything yet)

In [93]:
next_button = driver.find_element_by_partial_link_text("Next")
driver.execute_script("arguments[0].scrollIntoView(true)", next_button)
next_button.click()

### Wrapping with `while`

> Go back to the first page of results before you try to run this

You have a bunch of scraping code. It clicks the next button, then it stops. But you'd like it to go back up to the top! You can make that happen with a special `while` loop.

```python
while True:
    # Scrape your stuff
    # Click next button
```

This will go on FOREVER AND EVER until there is an error (when it can't find the Next button on the last page of results, you'll get an error).

- *Tip: Print out "Scraping a new page" every time you visit a new page, just to check that it's working*

In [99]:
while True:
    info_list = []
    nolink_list = []
    results = driver.find_elements_by_class_name("searchfieldtitle")
    for result in results:
        info_dic = {}
        name = result.find_elements_by_tag_name("td")[1].text
        try:
            url = result.find_elements_by_tag_name("td")[2].find_element_by_tag_name("a").get_attribute("href")
            info_dic["name"] = name
            info_dic["url"] = url
            info_list.append(info_dic)
        except:
            nolink_list.append(name)    
    
    next_button = driver.find_element_by_partial_link_text("Next")
    driver.execute_script("arguments[0].scrollIntoView(true)", next_button)
    next_button.click()


Scraping a new page
Scraping a new page
Scraping a new page
Scraping a new page
Scraping a new page
Scraping a new page


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"partial link text","selector":"Next"}
  (Session info: chrome=67.0.3396.87)
  (Driver info: chromedriver=2.40.565498 (ea082db3280dd6843ebfb08a625e3eb905c4f5ab),platform=Windows NT 10.0.17134 x86_64)


### Making it perfect

> Go back to the first page of results before you try to run this

Wrap all of your code in a `try`/`except` so that it doesn't finish with an error and you'll be good to go.

**Confirm your list has all of the vape shops in it.** If not, check where you are creating your empty list (`[]`) - if you do it in the wrong spot, it will overwrite your list every time you visit a page.

In [106]:
try:
    info_list = []
    nolink_list = []
    while True:
        results = driver.find_elements_by_class_name("searchfieldtitle")
        for result in results:
            info_dic = {}
            name = result.find_elements_by_tag_name("td")[1].text
            try:
                url = result.find_elements_by_tag_name("td")[2].find_element_by_tag_name("a").get_attribute("href")
                info_dic["name"] = name
                info_dic["url"] = url
                info_list.append(info_dic)
            except:
                nolink_list.append(name)    

        next_button = driver.find_element_by_partial_link_text("Next")
        driver.execute_script("arguments[0].scrollIntoView(true)", next_button)
        next_button.click()
        time.sleep(2)
except:
    pass

df = pd.DataFrame(info_list)
df

,name,url
0,VAPE IT STORE II,https://jportal.mdcourts.gov/license/pbLicense...
1,VAPE IT STORE I,https://jportal.mdcourts.gov/license/pbLicense...
2,VAPEPAD THE,https://jportal.mdcourts.gov/license/pbLicense...
3,VAPE LOFT THE,https://jportal.mdcourts.gov/license/pbLicense...
4,VAPE N CIGAR,https://jportal.mdcourts.gov/license/pbLicense...
5,VAPE DOJO,https://jportal.mdcourts.gov/license/pbLicense...
6,VAPORFI SOUTHSIDE,https://jportal.mdcourts.gov/license/pbLicense...
7,VAPE DO JO,https://jportal.mdcourts.gov/license/pbLicense...
8,VAPERTEKMD,https://jportal.mdcourts.gov/license/pbLicense...
9,VAPE PLUS,https://jportal.mdcourts.gov/license/pbLicense...


### Save this data as a csv

The filename should be `vape-shops-basic.csv`.

In [107]:
df.to_csv("vape-shops-basic.csv", index = False)

In [109]:
df1 = pd.read_csv("vape-shops-basic.csv")
df1

,name,url
0,VAPE IT STORE II,https://jportal.mdcourts.gov/license/pbLicense...
1,VAPE IT STORE I,https://jportal.mdcourts.gov/license/pbLicense...
2,VAPEPAD THE,https://jportal.mdcourts.gov/license/pbLicense...
3,VAPE LOFT THE,https://jportal.mdcourts.gov/license/pbLicense...
4,VAPE N CIGAR,https://jportal.mdcourts.gov/license/pbLicense...
5,VAPE DOJO,https://jportal.mdcourts.gov/license/pbLicense...
6,VAPORFI SOUTHSIDE,https://jportal.mdcourts.gov/license/pbLicense...
7,VAPE DO JO,https://jportal.mdcourts.gov/license/pbLicense...
8,VAPERTEKMD,https://jportal.mdcourts.gov/license/pbLicense...
9,VAPE PLUS,https://jportal.mdcourts.gov/license/pbLicense...


# Okay, let's scrape!

All right, get the actual data!

### Look at the URL of your first row

- *TIP: Remember `pd.set_option('display.max_colwidth', -1)` will let you see alllll of your strings*

In [117]:
pd.set_option('display.max_colwidth', -1)
df1["url"][1]

'https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=L3Gl%2FWyebeQ%3D'

### Use Selenium to visit that page

In [118]:
driver.get(df1["url"][1])

### Now, just like you did before, grab the additional data

You should probably save it into a dictionary! Don't try to put it into the dataframe yet, though. You want:

- Mailing address
- Location address
- License information (you can keep it as one field)
- Total amount paid
- Issued by
- If you're feeling crazy, get the licenses, too.

.

- *TIP: Licenses can be acquired by doing some really odd list slicing - think about where it starts and where it ends, relative to the beginning and end of everything.*
- *TIP: If you've gotten addicted to xpath, total amount paid and issued by might not work with it when doing other shops. You'll want to test it!*

In [127]:
v_dic = {}

# Mailing address
mailing = driver.find_elements_by_class_name("tablecelltext")[0].find_elements_by_tag_name("td")[0].text
v_dic["mailing"] = mailing

# Location address
location = driver.find_elements_by_class_name("tablecelltext")[1].find_elements_by_tag_name("td")[0].text
v_dic["location"] = location

# License information (you can keep it as one field)
license_info = driver.find_elements_by_class_name("tablecelltext")[0].find_elements_by_tag_name("td")[1].text
v_dic["license_info"] = license_info
    
# Total amount paid
a_paid = driver.find_elements_by_class_name("tablecelltext")[-3].find_elements_by_tag_name("td")[1].text
v_dic["total amount paid"] = a_paid

# Issued by
issued_by = driver.find_elements_by_class_name("tablecelltext")[-2].text
v_dic["issued by"] = issued_by

# If you're feeling crazy, get the licenses, too.
license_detail = []
licenses = driver.find_elements_by_tag_name("table")[-4].find_elements_by_class_name("tablecelltext")
for license in licenses[:-3]:
    license_d = license.text.split(" ")[2:-2]
    license_c = " ".join(license_d)
    license_detail.append(license_c)
v_dic["license_detail"] = license_detail

print("here is my dictionary")
v_dic

here is my dictionary


{'mailing': 'AMIN NARGIS\n1104 PLANTERS PLACE\nSALISBURY, MD 21804',
 'location': 'VAPE IT STORE I\n1724 N SALISBURY BLVD UNIT 2\nSALISBURY, MD 21801',
 'license_info': 'License Status: Issued\nLicense No.: 22375605\nControl No.: 22591855\nDate of Issue: 4/27/2018\nMonths Paid: 12\nExp. Date: 4/30/2019\nSubdivision: 22 Salisbury',
 'total amount paid': '$ 24.00',
 'issued by': 'WICOMICO COUNTY, CLERK OF CIRCUIT COURT\nP.O. BOX 198\nSALISBURY, MARYLAND 21803-0198',
 'license_detail': ["TRADER'S LICENSE", 'CHAIN STORE']}

### Move all of this into one cell

It should visit the URL, then grab the data and put it into a dictionary.

In [131]:
driver.get(df1["url"][1])

v_dic = {}

# Mailing address
mailing = driver.find_elements_by_class_name("tablecelltext")[0].find_elements_by_tag_name("td")[0].text
v_dic["mailing"] = mailing

# Location address
location = driver.find_elements_by_class_name("tablecelltext")[1].find_elements_by_tag_name("td")[0].text
v_dic["location"] = location

# License information (you can keep it as one field)
license_info = driver.find_elements_by_class_name("tablecelltext")[0].find_elements_by_tag_name("td")[1].text
v_dic["license_info"] = license_info
    
# Total amount paid
a_paid = driver.find_elements_by_class_name("tablecelltext")[-3].find_elements_by_tag_name("td")[1].text
v_dic["total amount paid"] = a_paid

# Issued by
issued_by = driver.find_elements_by_class_name("tablecelltext")[-2].text
v_dic["issued by"] = issued_by

# If you're feeling crazy, get the licenses, too.
license_detail = []
licenses = driver.find_elements_by_tag_name("table")[-4].find_elements_by_class_name("tablecelltext")
for license in licenses[:-3]:
    license_d = license.text.split(" ")[2:-2]
    license_c = " ".join(license_d)
    license_detail.append(license_c)
v_dic["license_detail"] = license_detail

print("here is my dictionary")
v_dic

here is my dictionary


{'mailing': 'AMIN NARGIS\n1104 PLANTERS PLACE\nSALISBURY, MD 21804',
 'location': 'VAPE IT STORE I\n1724 N SALISBURY BLVD UNIT 2\nSALISBURY, MD 21801',
 'license_info': 'License Status: Issued\nLicense No.: 22375605\nControl No.: 22591855\nDate of Issue: 4/27/2018\nMonths Paid: 12\nExp. Date: 4/30/2019\nSubdivision: 22 Salisbury',
 'total amount paid': '$ 24.00',
 'issued by': 'WICOMICO COUNTY, CLERK OF CIRCUIT COURT\nP.O. BOX 198\nSALISBURY, MARYLAND 21803-0198',
 'license_detail': ["TRADER'S LICENSE", 'CHAIN STORE']}

### Change it into a function

You'll want to have this function accept a `row`, and send back a `pd.Series`. You can just use `pd.Series(your_dictionary)` (but it better have a better name than `your_dictionary`!).

- *TIP: Make sure you `return` something!*
- *TIP: Make sure you change everything to reflect the row's url, not the URL you typed in*

In [132]:
def get_shop_info(row):
    driver.get(row["url"])
    
    # Mailing address
    mailing = driver.find_elements_by_class_name("tablecelltext")[0].find_elements_by_tag_name("td")[0].text
    # Location address
    location = driver.find_elements_by_class_name("tablecelltext")[1].find_elements_by_tag_name("td")[0].text
    # License information (you can keep it as one field)
    license_info = driver.find_elements_by_class_name("tablecelltext")[0].find_elements_by_tag_name("td")[1].text
    # Total amount paid
    a_paid = driver.find_elements_by_class_name("tablecelltext")[-3].find_elements_by_tag_name("td")[1].text
    # Issued by
    issued_by = driver.find_elements_by_class_name("tablecelltext")[-2].text
    # If you're feeling crazy, get the licenses, too.
    license_detail = []
    licenses = driver.find_elements_by_tag_name("table")[-4].find_elements_by_class_name("tablecelltext")
    for license in licenses[:-3]:
        license_d = license.text.split(" ")[2:-2]
        license_c = " ".join(license_d)
        license_detail.append(license_c)
    
    return pd.Series({
        "mailing" : mailing,
        "location" : location,
        "license_info" : license_info,
        "total amount paid": a_paid,
        "issued by" : issued_by,
        "license_detail": license_detail
    })


### Use your dataframe and `.apply` to pull all of the data from the vape shops

Once you know it's working, use the whole 

- *TIP: Try using it with `.head(3)` first*
- *TIP: You'll want to use `.apply` with your new function*
- *TIP: Issued By and Total Paid are goign to give you problems if you tried to use xpath! Try checking the classes and think about find_elementSSSSS and working backwards instead of forwards.*
- *TIP: You might need a `try`/`except`*
- *TIP: Make sure you're using `axis=1`*
- *TIP: Use `.join` the big thing with all of the `dfs` - make sure you name them right!*

In [134]:
df_with_info = df1.head(3).apply(get_shop_info, axis = 1).join(df1)

df_with_info

,mailing,location,license_info,total amount paid,issued by,license_detail,name,url
0,"AMIN NARGIS\n1104 PLANTERS PLACE\nSALISBURY, MD 21804","VAPE IT STORE II\n1015 S SALISBURY BLVD\nSALISBURY, MD 21801",License Status: Issued\nLicense No.: 22375606\nControl No.: 22884439\nDate of Issue: 4/27/2018\nMonths Paid: 12\nExp. Date: 4/30/2019\nSubdivision: 22 Salisbury,$ 41.00,"WICOMICO COUNTY, CLERK OF CIRCUIT COURT\nP.O. BOX 198\nSALISBURY, MARYLAND 21803-0198","[OTHER TOBACCO PROD RETLR, CHAIN STORE, TRADER'S LICENSE]",VAPE IT STORE II,https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=HT0XjAXVv8M%3D
1,"AMIN NARGIS\n1104 PLANTERS PLACE\nSALISBURY, MD 21804","VAPE IT STORE I\n1724 N SALISBURY BLVD UNIT 2\nSALISBURY, MD 21801",License Status: Issued\nLicense No.: 22375605\nControl No.: 22591855\nDate of Issue: 4/27/2018\nMonths Paid: 12\nExp. Date: 4/30/2019\nSubdivision: 22 Salisbury,$ 24.00,"WICOMICO COUNTY, CLERK OF CIRCUIT COURT\nP.O. BOX 198\nSALISBURY, MARYLAND 21803-0198","[TRADER'S LICENSE, CHAIN STORE]",VAPE IT STORE I,https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=L3Gl%2FWyebeQ%3D
2,"ANJ DISTRIBUTIONS LLC\n2504 ORCHARD KNOLL WAY\nODENTON, MD 21113","VAPEPAD THE\n2299 JOHNS HOPKINS ROAD\nGAMBRILLS, MD 21054",License Status: Issued\nLicense No.: 02304705\nControl No.: 02685930\nDate of Issue: 4/05/2018\nMonths Paid: 12\nExp. Date: 4/30/2019\nSubdivision: 02 Anne Arundel County,$ 94.00,"ROBERT P. DUCKWORTH, CLERK OF CIRCUIT COURT\n8 CHURCH CIRCLE, ROOM H-101\nANNAPOLIS, MARYLAND 21401","[TRADER'S LICENSE, E-NICOTINE DLVY SYS RETLR]",VAPEPAD THE,https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=3uEmL1HbRcs%3D


## Save as `vape-total.csv`

Make sure you don't save the index! Open it up in a text editor or Excel to make sure it's correct.

In [135]:
df_with_info = df1.apply(get_shop_info, axis = 1).join(df1)

df_with_info

,mailing,location,license_info,total amount paid,issued by,license_detail,name,url
0,"AMIN NARGIS\n1104 PLANTERS PLACE\nSALISBURY, MD 21804","VAPE IT STORE II\n1015 S SALISBURY BLVD\nSALISBURY, MD 21801",License Status: Issued\nLicense No.: 22375606\nControl No.: 22884439\nDate of Issue: 4/27/2018\nMonths Paid: 12\nExp. Date: 4/30/2019\nSubdivision: 22 Salisbury,$ 41.00,"WICOMICO COUNTY, CLERK OF CIRCUIT COURT\nP.O. BOX 198\nSALISBURY, MARYLAND 21803-0198","[OTHER TOBACCO PROD RETLR, CHAIN STORE, TRADER'S LICENSE]",VAPE IT STORE II,https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=HT0XjAXVv8M%3D
1,"AMIN NARGIS\n1104 PLANTERS PLACE\nSALISBURY, MD 21804","VAPE IT STORE I\n1724 N SALISBURY BLVD UNIT 2\nSALISBURY, MD 21801",License Status: Issued\nLicense No.: 22375605\nControl No.: 22591855\nDate of Issue: 4/27/2018\nMonths Paid: 12\nExp. Date: 4/30/2019\nSubdivision: 22 Salisbury,$ 24.00,"WICOMICO COUNTY, CLERK OF CIRCUIT COURT\nP.O. BOX 198\nSALISBURY, MARYLAND 21803-0198","[TRADER'S LICENSE, CHAIN STORE]",VAPE IT STORE I,https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=L3Gl%2FWyebeQ%3D
2,"ANJ DISTRIBUTIONS LLC\n2504 ORCHARD KNOLL WAY\nODENTON, MD 21113","VAPEPAD THE\n2299 JOHNS HOPKINS ROAD\nGAMBRILLS, MD 21054",License Status: Issued\nLicense No.: 02304705\nControl No.: 02685930\nDate of Issue: 4/05/2018\nMonths Paid: 12\nExp. Date: 4/30/2019\nSubdivision: 02 Anne Arundel County,$ 94.00,"ROBERT P. DUCKWORTH, CLERK OF CIRCUIT COURT\n8 CHURCH CIRCLE, ROOM H-101\nANNAPOLIS, MARYLAND 21401","[TRADER'S LICENSE, E-NICOTINE DLVY SYS RETLR]",VAPEPAD THE,https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=3uEmL1HbRcs%3D
3,DISBROW II EMERSON HARRINGTON,"VAPE LOFT THE\n185 MITCHELLS CHANCE RD\nEDGEWATER, MD 21037",License Status: Issued\nLicense No.: 02310799\nControl No.: 02686069\nDate of Issue: 4/03/2018\nMonths Paid: 12\nExp. Date: 4/30/2019\nSubdivision: 02 Anne Arundel County,$ 154.00,"ROBERT P. DUCKWORTH, CLERK OF CIRCUIT COURT\n8 CHURCH CIRCLE, ROOM H-101\nANNAPOLIS, MARYLAND 21401","[TRADER'S LICENSE, VAPE SHOP VENDOR]",VAPE LOFT THE,https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=csrXcvFJXmY%3D
4,DISCOUNT TOBACCO ESSEX LLC,"VAPE N CIGAR\n7104 MINSTREL UNIT #7\nCOLUMBIA, MD 21045",License Status: Issued\nLicense No.: 13343011\nControl No.: 13856368\nDate of Issue: 4/30/2018\nMonths Paid: 12\nExp. Date: 4/30/2019\nSubdivision: 13 Howard County,$ 84.00,"WAYNE A. ROBEY, CLERK OF CIRCUIT COURT\n9250 BENDIX ROAD\nCOLUMBIA, MARYLAND 21045","[TRADER'S LICENSE, OTHER TOBACCO PROD RETLR]",VAPE N CIGAR,https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=xKwydFAY6Hs%3D
5,FAIRGROUND VILLAGE LLC,"VAPE DOJO\n330 ONE FORTY VILLAGE ROAD\nUNIT 15\nWESTMINSTER, MD 21157",License Status: Issued\nLicense No.: 06327188\nControl No.: 06946760\nDate of Issue: 4/05/2018\nMonths Paid: 12\nExp. Date: 4/30/2019\nSubdivision: 06 Westminster,$ 179.00,"DONALD B. SEALING II, CLERK OF CIRCUIT COURT\n55 NORTH COURT STREET\nWESTMINSTER, MARYLAND 21157-5155","[TRADER'S LICENSE, VAPE SHOP VENDOR]",VAPE DOJO,https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=dwg%2FY3M8nQU%3D
6,JENS VAPOR LLC,"VAPORFI SOUTHSIDE\n853 EAST FORT AVE\nBALTIMORE, MD 21230",License Status: Issued\nLicense No.: 30390631\nControl No.: 30190160\nDate of Issue: 4/20/2018\nMonths Paid: 12\nExp. Date: 4/30/2019\nSubdivision: 30 Baltimore City,$ 189.00,"MARILYN BENTLEY, CLERK CIRCUIT COURT\n100 N. CALVERT STREET, ROOM 627\nBALTIMORE, MARYLAND 21202","[TRADER'S LICENSE, VAPE SHOP VENDOR]",VAPORFI SOUTHSIDE,https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=wqeEvS2LrNQ%3D
7,MONROE LLC,"VAPE DO JO\n2021 LIBERTY ROAD\nSYKESVILLE, MD 21784",License Status: Issued\nLicense No.: 06328812\nControl No.: 06328812\nDate of Issue: 5/03/2018\nMonths Paid: 12\nExp. Date: 4/30/2019\nSubdivision: 06 Sykesville,$ 154.00,"DONALD B. SEALING II, CLERK OF CIRCUIT COURT\n55 NORTH COURT STREET\nWESTMINSTER, MARYLAND 21157-5155","[TRADER'S LICEN

In [136]:
df_with_info.to_csv("vape-total.csv", index = False)